In [13]:
import os
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI

# Nuevas importaciones para la sintaxis moderna (LCEL)
from langchain_core.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import  RunnablePassthrough
from pathlib import Path

In [14]:
# --- Carga de la API key (esto está perfecto) ---
try:
    with open("../gem_apikey.txt") as f:
        api_key = f.read().strip()
    print("API key cargada correctamente.")
except FileNotFoundError:
    print("Error: No se encontró el archivo '../gem_apikey.txt'.")
    api_key = None

API key cargada correctamente.


In [15]:
def get_available_indexes(index_folder):
    """
    Busca en la carpeta de índices y devuelve una lista de los disponibles.
    """    
    if not os.path.exists(index_folder):
        print(f"Error: La carpeta de índices '{index_folder}' no fue encontrada.")
        return []

    # Obtener lista de directorios de índices (ej: 'faiss_index_doc1')
    processed_folders = [f.name for f in Path(index_folder).iterdir() if f.is_dir()]
    
    return processed_folders

In [16]:
def start_chat():
    """
    Presenta un menú para elegir un documento, carga su base de datos vectorial
    y permite chatear con una personalidad dinámica.
    """
    if not api_key:
        print("Proceso detenido. No se pudo cargar la API key.")
        return

    # --- CONFIGURACIÓN DE LA ESTRUCTURA DE CARPETAS FINAL ---
    FOLDER_BASE = "../database"
    FOLDER_INDEX = os.path.join(FOLDER_BASE, "indexed", "faiss")

    # 1. OBTENER Y MOSTRAR LOS ÍNDICES DISPONIBLES
    index_list = get_available_indexes(FOLDER_INDEX)

    if not index_list:
        print("No se encontraron documentos procesados para iniciar el chat.")
        return

    teachers_data = [
        {"teacher": "Victor", "docs": 0},
        {"teacher": "Diego", "docs": len(index_list)},
        {"teacher": "Javier", "docs": 0},
        {"teacher": "Luis", "docs": 0},
    ]

    print("\n--- Profesores Disponibles ---")
    for i, item in enumerate(teachers_data):
        status = "✅ Disponible" if item["docs"] > 0 else "⛔ No disponible"
        print(f"{i + 1}. {item['teacher']} ({item['docs']} documentos) {status}")

    # Selección del profesor
    selected_index = -1
    while selected_index < 0 or selected_index >= len(teachers_data):
        try:
            choice = input(f"\nSelecciona un profesor (1-{len(teachers_data)}): ")
            selected_index = int(choice) - 1
            if selected_index < 0 or selected_index >= len(teachers_data):
                print("Selección inválida.")
        except ValueError:
            print("Entrada no válida.")

    selected_teacher = teachers_data[selected_index]

    if selected_teacher["docs"] == 0:
        print(
            f"\nEl profesor {selected_teacher['teacher']} aún no tiene temas disponibles."
        )
        return

    print(
        f"\nHas seleccionado al profesor {selected_teacher['teacher']} con especialidad en {selected_teacher['docs']} areas de conocimiento."
    )
    # Aquí podrías continuar con la selección de área o documento específico

    print("\n--- Áreas de conocimiento disponibles para Consulta ---")
    # Limpiamos los nombres para que sean más legibles en el menú
    friendly_names = [name.replace("faiss_index_", "") for name in index_list]

    for i, name in enumerate(friendly_names):
        print(f"{i + 1}. {name}")

    # 2. PERMITIR AL USUARIO ELEGIR UN DOCUMENTO
    user_selection = -1
    while user_selection < 0 or user_selection >= len(index_list):
        try:
            choice = input(
                f"\nElige el número del área de conocimiento sobre la cual desea profundizar (1-{len(index_list)}): "
            )
            user_selection = int(choice) - 1
            if user_selection < 0 or user_selection >= len(index_list):
                print("Selección inválida. Por favor, elige un número de la lista.")
        except ValueError:
            print("Entrada no válida. Por favor, introduce un número.")

    selected_folder_name = index_list[user_selection]
    selected_document_name = friendly_names[user_selection]
    full_index_path = os.path.join(FOLDER_INDEX, selected_folder_name)

    # 3. PERSONALIZACIÓN DINÁMICA DE LA PERSONALIDAD
    print("\n--- Configuración del Asistente ---")
    teacher_name = selected_teacher["teacher"]

    knowledge_domain = selected_document_name

    # 4. Carga de la base de datos vectorial SELECCIONADA
    print(
        f"\nCargando la base de datos de '{selected_document_name}' desde '{full_index_path}'..."
    )

    print(f"Profesor '{teacher_name}' es experto en '{knowledge_domain}' ")
    try:
        embeddings = GoogleGenerativeAIEmbeddings(
            model="models/embedding-001", google_api_key=api_key
        )
        vectorstore = FAISS.load_local(
            full_index_path, embeddings, allow_dangerous_deserialization=True
        )
        retriever = vectorstore.as_retriever()
        print("¡Base de datos cargada exitosamente!")
    except Exception as e:
        print(f"Error al cargar la base de datos vectorial: {e}")
        return

    # 5. Configuración del LLM y la cadena RAG
    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash", temperature=0.3, google_api_key=api_key
    )

    # 6.1 Prompt con personalidad y contexto
    system_tamplate = """Eres un asistente virtual que se comporta como un profesor de {knowledgeDomain} experto.
    Tu nombre es "IAsistente de {Teacher}". Eres amable, didáctico y te encanta {knowledgeDomain}. 
    REGLA ESTRICTA: Solo puedes responder preguntas relacionadas con {knowledgeDomain} basándote en el contexto proporcionado. Si la pregunta no está relacionada con estos temas, 
    responde: "Lo siento, mi especialidad es {knowledgeDomain}. No puedo responder preguntas sobre otros temas.
    Cuando te digan "salir", te despedirás amablemente."""

    system_message_prompt = SystemMessagePromptTemplate.from_template(system_tamplate)

    # 6.2 Prompt para la pregunta del usuario
    human_template = "{user_question}"
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

    # 6.3 Combinación de los prompts en un ChatPromptTemplate

    chat_prompt_template = ChatPromptTemplate.from_messages(
        [system_message_prompt, human_message_prompt]
    )

    # 6.4 Definición de las variables del sistema
    chat_prompt_template = chat_prompt_template.partial(
        Teacher=teacher_name, knowledgeDomain=knowledge_domain
    )

    rag_chain = (
        {"context": retriever, "user_question": RunnablePassthrough()}
        | chat_prompt_template
        | llm
        | StrOutputParser()
    )

    print(
        f"\n¡El Asistente de {teacher_name} está listo para responder preguntas sobre {knowledge_domain}!"
    )
    print("Escribe 'salir' para terminar la conversación.")

    # 7. Bucle de chat
    while True:
        query = input("\nPregunta del estudiante: ")

        response = rag_chain.invoke(query)

        print(f"\nRespuesta del Asistente de {teacher_name}:")
        print(response)

        if query.lower() == "salir":
            break

if __name__ == "__main__":
    start_chat()


--- Profesores Disponibles ---
1. Victor (0 documentos) ⛔ No disponible
2. Diego (3 documentos) ✅ Disponible
3. Javier (0 documentos) ⛔ No disponible
4. Luis (0 documentos) ⛔ No disponible



Selecciona un profesor (1-4):  2



Has seleccionado al profesor Diego con especialidad en 3 areas de conocimiento.

--- Áreas de conocimiento disponibles para Consulta ---
1. sistema-cardiovascular
2. sistema-nervioso-central
3. sistema-oseo



Elige el número del área de conocimiento sobre la cual desea profundizar (1-3):  3



--- Configuración del Asistente ---

Cargando la base de datos de 'sistema-oseo' desde '../database\indexed\faiss\faiss_index_sistema-oseo'...
Profesor 'Diego' es experto en 'sistema-oseo' 
¡Base de datos cargada exitosamente!

¡El Asistente de Diego está listo para responder preguntas sobre sistema-oseo!
Escribe 'salir' para terminar la conversación.


...


Has seleccionado al profesor Diego con especialidad en 3 areas de conocimiento.

--- Áreas de conocimiento disponibles para Consulta ---
1. sistema-cardiovascular
2. sistema-nervioso-central
3. sistema-oseo



Elige el número del área de conocimiento sobre la cual desea profundizar (1-3):  3



--- Configuración del Asistente ---

Cargando la base de datos de 'sistema-oseo' desde '../database\indexed\faiss\faiss_index_sistema-oseo'...
Profesor 'Diego' es experto en 'sistema-oseo' 
¡Base de datos cargada exitosamente!

¡El Asistente de Diego está listo para responder preguntas sobre sistema-oseo!
Escribe 'salir' para terminar la conversación.



Pregunta del estudiante:  que es el sistema cardiovascular?


KeyError: "Input to ChatPromptTemplate is missing variables {'pregunta_usuario'}.  Expected: ['pregunta_usuario'] Received: ['context', 'user_question']\nNote: if you intended {pregunta_usuario} to be part of the string and not a variable, please escape it with double curly braces like: '{{pregunta_usuario}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "